In [76]:
from IPython.display import IFrame
IFrame("homework1.pdf", width=600, height=300)

In [77]:
#importing libraries
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")


In [78]:
#utility function for parsing JSONdata
def parseData(fname):
    for l in fname:
        yield eval(l)

In [79]:
file = open('beer_50000.json')

In [80]:
# loading and parsing JSONdata
data  =list(parseData(file))

In [81]:
data[1]

{'beer/ABV': 6.2,
 'beer/beerId': '48213',
 'beer/brewerId': '10325',
 'beer/name': 'Red Moon',
 'beer/style': 'English Strong Ale',
 'review/appearance': 3.0,
 'review/aroma': 2.5,
 'review/overall': 3.0,
 'review/palate': 3.0,
 'review/taste': 3.0,
 'review/text': 'Dark red color, light beige foam, average.\tIn the smell malt and caramel, not really light.\tAgain malt and caramel in the taste, not bad in the end.\tMaybe a note of honey in teh back, and a light fruitiness.\tAverage body.\tIn the aftertaste a light bitterness, with the malt and red fruit.\tNothing exceptional, but not bad, drinkable beer.',
 'review/timeStruct': {'hour': 13,
  'isdst': 0,
  'mday': 1,
  'min': 44,
  'mon': 3,
  'sec': 57,
  'wday': 6,
  'yday': 60,
  'year': 2009},
 'review/timeUnix': 1235915097,
 'user/profileName': 'stcules'}


# Tasks — Regression (week 1):

# 1.No of Reviews for each style of beer in the dataset (`beer/style`) ,  
#  avg value of `review/taste` for reviews from each style  

In [82]:
def avgReview(data):
    totalRev = dict()
    revFreq = dict()
    avgRev = dict()
    for dict_ in data:
        if dict_['beer/style'] not in totalRev:
            totalRev[dict_['beer/style']] = 0
            revFreq[dict_['beer/style']]  = 0
            
        totalRev[dict_['beer/style']] += dict_['review/taste']
        revFreq [dict_['beer/style']] += 1
    for style in totalRev:
        avgRev[style] = round(totalRev[style]/revFreq[style], 6)
    return avgRev, revFreq

In [83]:
avgRev ,revFreq = avgReview(data)

In [84]:
revFreq

{'Altbier': 165,
 'American Adjunct Lager': 242,
 'American Amber / Red Ale': 665,
 'American Amber / Red Lager': 42,
 'American Barleywine': 825,
 'American Black Ale': 138,
 'American Blonde Ale': 357,
 'American Brown Ale': 314,
 'American Dark Wheat Ale': 14,
 'American Double / Imperial IPA': 3886,
 'American Double / Imperial Pilsner': 14,
 'American Double / Imperial Stout': 5964,
 'American IPA': 4113,
 'American Malt Liquor': 90,
 'American Pale Ale (APA)': 2288,
 'American Pale Lager': 123,
 'American Pale Wheat Ale': 154,
 'American Porter': 2230,
 'American Stout': 591,
 'American Strong Ale': 166,
 'American Wild Ale': 98,
 'Baltic Porter': 514,
 'Belgian Dark Ale': 175,
 'Belgian IPA': 128,
 'Belgian Pale Ale': 144,
 'Belgian Strong Dark Ale': 146,
 'Belgian Strong Pale Ale': 632,
 'Berliner Weissbier': 10,
 'BiÃ¨re de Garde': 7,
 'Black & Tan': 122,
 'Bock': 148,
 'Braggot': 26,
 'California Common / Steam Beer': 11,
 'Chile Beer': 11,
 'Cream Ale': 69,
 'Czech Pilsener'

In [85]:
avgRev

{'Altbier': 3.40303,
 'American Adjunct Lager': 2.948347,
 'American Amber / Red Ale': 3.513534,
 'American Amber / Red Lager': 3.690476,
 'American Barleywine': 4.064242,
 'American Black Ale': 3.873188,
 'American Blonde Ale': 3.254902,
 'American Brown Ale': 3.743631,
 'American Dark Wheat Ale': 3.678571,
 'American Double / Imperial IPA': 4.033325,
 'American Double / Imperial Pilsner': 3.821429,
 'American Double / Imperial Stout': 4.479963,
 'American IPA': 4.000851,
 'American Malt Liquor': 2.255556,
 'American Pale Ale (APA)': 3.649694,
 'American Pale Lager': 3.215447,
 'American Pale Wheat Ale': 3.334416,
 'American Porter': 4.081839,
 'American Stout': 3.819797,
 'American Strong Ale': 3.569277,
 'American Wild Ale': 4.188776,
 'Baltic Porter': 4.213035,
 'Belgian Dark Ale': 3.34,
 'Belgian IPA': 3.949219,
 'Belgian Pale Ale': 3.739583,
 'Belgian Strong Dark Ale': 3.695205,
 'Belgian Strong Pale Ale': 4.056171,
 'Berliner Weissbier': 3.55,
 'BiÃ¨re de Garde': 3.928571,
 'Bla

# 2. Simple predictor with a single binary feature indicating whether a beer is an `American IPA`

In [86]:
def genFeature(data):
    X = []
    y = []
    for d in data:
        if d['beer/style'] == 'American IPA':
            X.append([1, 1])
        else:
            X.append([1, 0])
        y.append(d['review/taste'])
    return X, y

In [87]:
X, y = genFeature(data)

In [88]:
X = np.array(X)
y = np.array(y)


Linear Regression using numpy



Linear Regression using numpy OLS



In [89]:
theta, residuals, rank, s = np.linalg.lstsq(X, y)

In [90]:
theta[0], theta[1]


(3.9152047420838927, 0.08564621828574305)

In [91]:
residuals

array([25650.80867309])



Linear Regression using sckit-learn



In [92]:
reg = LinearRegression()
reg.fit(X[:, 1:], y)
y_pred = reg.predict(X[:, 1:])

print("coeficients:\t",reg.coef_)
print("intercept:\t",reg.intercept_)
print("MSE :\t",MSE(y, y_pred ))
print("r2_score :\t",r2_score(y, y_pred ))


coeficients:	 [0.08564622]
intercept:	 3.9152047420838145
MSE :	 0.5130161734617366
r2_score :	 0.0010782641230113743


theta[0] represent the y-intercept of regression line

theta[1] represent the slope of regression line

# 3.  Splitting the data into train-test sets , Finding MSE on training and testing set

In [93]:
X.shape

(50000, 2)

In [94]:
y.shape

(50000,)

In [95]:
X_train, X_test = X[:int(len(X)/2),1:], X[int(len(X)/2):,1:]

In [96]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]



Using normal equations






Using scikit-learn

In [97]:
# linear Regression
reg = LinearRegression()
reg.fit(X_train, y_train)
print("coeficients: ",reg.coef_)
print("intercept: ",reg.intercept_)
print("MSE on train set: ",MSE(y_train, reg.predict(X_train)) )
print("MSE on test set: ",MSE(y_test, reg.predict(X_test)))
print("r2_score on train set:",r2_score(y_train, reg.predict(X_train)) )
print("r2_score on test set:",r2_score(y_test, reg.predict(X_test)) )

coeficients:  [0.05606027]
intercept:  3.9043563922942206
MSE on train set:  0.558107286558669
MSE on test set:  0.46841005096664573
r2_score on train set: 0.0004442931134949202
r2_score on test set: 7.227980262203282e-05


# 4. Extending the above model , incorporating binary features for every style of beer with ≥ 50 reviews

In [98]:
def genFeatures(data, revFreq):
    beer_rev50 = []
    for beer_style in revFreq.keys():
        if revFreq[beer_style] > 50:
            beer_rev50.append(beer_style)

    #print( "beer styles with reviews count > 50 are :",len(beer_rev50))
    X = []
    y = []
    for d in data:
        x = []
        for beer in beer_rev50:
            x.append( d['beer/style'] == beer )
        X.append(x)
        y.append(d['review/taste'])
    return X, y

In [99]:
X, y = genFeatures(data, revFreq)

In [100]:
X = np.array(X)
y = np.array(y)

In [101]:
X.shape, y.shape

((50000, 74), (50000,))

In [102]:
X_train, X_test = X[:int(len(X)/2),:], X[int(len(X)/2):,:]

In [103]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]




Multiple Linear Regression Using scikit-learn



In [104]:
# linear Regression
reg = LinearRegression()
reg.fit(X_train, y_train)

print("coeficients: ",reg.coef_)
print("intercept: ",reg.intercept_)

print("MSE on training set: ",MSE(y_train, reg.predict(X_train)) )
print("MSE on testing set: ",MSE(y_test, reg.predict(X_test)) )

print("r2_score on train set:",r2_score(y_train, reg.predict(X_train)) )
print("r2_score on test set:",r2_score(y_test, reg.predict(X_test)) )

coeficients:  [ 0.71136364  0.09815076  0.12193999 -0.4664673   0.02739474  0.12395105
  0.22651515 -0.0058248  -0.2937747   0.01699134 -0.50681818  0.58195733
  0.76540404 -0.83277972  0.14466111  0.11433566  0.39318182  0.26818182
  0.25681818 -0.92019846 -0.02450111  0.69564175 -0.1798951   0.26709486
 -0.10223103  0.01818182  0.45638407 -0.58598485  0.10049889 -0.13106061
 -0.45410882  0.38356643  0.33596789  0.48544372  0.3763751  -0.65227273
  0.44318182 -0.35681818  0.35359848  0.60472028 -0.09003966 -0.22045455
 -0.17824675  0.26761104 -0.11634199  0.1527972   0.16385851 -0.41285266
  0.03420746  0.19621212  0.33580477 -0.74318182  0.20312334  0.8416167
 -0.10681818 -0.73802386 -0.18884943 -0.63806818  0.13786267  0.05895722
  0.3305986   0.12651515 -0.31931818 -1.00681818  0.20984848  0.65508658
 -0.20681818 -0.38622995  0.46842454  0.11320382  0.3449362  -0.27061129
 -0.62765152 -1.23390152]
intercept:  3.606818181818188
MSE on training set:  0.3678402770901444
MSE on testing

# Tasks — Classification (week 2):

# 5. A predictor that estimates whether a beer is an `American IPA` using two features:
[`beer/ABV`,`review/taste`]

In [105]:
def feature_ABV_taste(datum):
    return [1, datum['beer/ABV'], datum['review/taste']]

In [106]:
X = [feature_ABV_taste(d) for d in data]
y = [d['beer/style'] == 'American IPA' for d in data]

In [107]:
len(X), len(y)

(50000, 50000)

In [108]:
X_train, X_test = X[:int(len(X)/2)], X[int(len(X)/2):]

In [109]:
y_train, y_test = y[:int(len(y)/2)], y[int(len(y)/2):]

In [110]:
clf = svm.SVC(C=1000)
clf.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [111]:
y_pred = clf.predict(X_train)
train_acc = accuracy_score(y_train, y_pred )
print("Train set accuracy score : ", train_acc )
print("confusion matrix :\n", confusion_matrix(y_train, y_pred))
print("classification report :\n",classification_report(y_train, y_pred))

Train set accuracy score :  0.9226
confusion matrix :
 [[22594   246]
 [ 1689   471]]
classification report :
              precision    recall  f1-score   support

      False       0.93      0.99      0.96     22840
       True       0.66      0.22      0.33      2160

avg / total       0.91      0.92      0.90     25000



In [112]:
y_pred = clf.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print("Test set accuracy score :", test_acc )
print("confusion matrix : \n",confusion_matrix(y_test, y_pred))
print("classification report :\n",classification_report(y_test, y_pred))

Test set accuracy score : 0.85632
confusion matrix : 
 [[21364  1683]
 [ 1909    44]]
classification report :
              precision    recall  f1-score   support

      False       0.92      0.93      0.92     23047
       True       0.03      0.02      0.02      1953

avg / total       0.85      0.86      0.85     25000



# 6. Feature Engineering using common words in `review/text` of   `American IPA`

In [113]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [114]:
textRevAmericanIPA = []
for d in data:
    if d['beer/style'] == 'American IPA':
        textRevAmericanIPA.append(d['review/text'])


In [115]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = []
for text in textRevAmericanIPA:
    tokens += tokenizer.tokenize(text)
    

In [116]:
stop_words = set(stopwords.words('english'))
all_words = [w for w in tokens if not w in stop_words and len(w) > 2]

In [117]:
freqDist = nltk.FreqDist(all_words)

In [118]:
freqDist.most_common(50)

[('hops', 4686),
 ('head', 3942),
 ('The', 3872),
 ('beer', 3804),
 ('IPA', 3663),
 ('hop', 3343),
 ('malt', 3117),
 ('nice', 2959),
 ('citrus', 2865),
 ('good', 2613),
 ('one', 2040),
 ('orange', 1916),
 ('like', 1916),
 ('white', 1876),
 ('carbonation', 1825),
 ('color', 1799),
 ('glass', 1799),
 ('taste', 1772),
 ('This', 1760),
 ('grapefruit', 1754),
 ('well', 1745),
 ('bitterness', 1734),
 ('pine', 1700),
 ('bitter', 1695),
 ('bit', 1690),
 ('flavor', 1684),
 ('lacing', 1660),
 ('light', 1660),
 ('finish', 1543),
 ('aroma', 1541),
 ('sweet', 1508),
 ('little', 1448),
 ('amber', 1339),
 ('caramel', 1319),
 ('body', 1292),
 ('Very', 1235),
 ('medium', 1179),
 ('bottle', 1151),
 ('Pours', 1141),
 ('floral', 1132),
 ('hoppy', 1102),
 ('great', 1048),
 ('nose', 1044),
 ('really', 1036),
 ('balanced', 1029),
 ('notes', 1014),
 ('smell', 951),
 ('dry', 926),
 ('fresh', 915),
 ('would', 909)]

feat = ['citrus', 'IPA', 'bitter', 'grapefruit','caramel']


In [119]:
def customFeatures(datum):
    return [1, 
            datum['beer/ABV'],
           'citrus' in datum['review/text'],
           'IPA' in datum['review/text'],
           'bitter' in datum['review/text'],
           'grapefruit' in datum['review/text'],
           'caramel' in datum['review/text']]

In [120]:
X = [customFeatures(d) for d in data]
y = [d['beer/style'] == 'American IPA' for d in data]

In [121]:
X_train, X_test = X[:int(len(X)/2)], X[int(len(X)/2):]

In [122]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]

In [123]:
clf2 = svm.SVC(C=1000)
clf2.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [124]:
y_pred = clf2.predict(X_train)
train_acc = accuracy_score(y_train, y_pred )

print("Train set accuracy score : ", train_acc )
print("confusion matrix :\n", confusion_matrix(y_train, y_pred))
print("classification report :\n",classification_report(y_train, y_pred))

Train set accuracy score :  0.96248
confusion matrix :
 [[22353   487]
 [  451  1709]]
classification report :
              precision    recall  f1-score   support

      False       0.98      0.98      0.98     22840
       True       0.78      0.79      0.78      2160

avg / total       0.96      0.96      0.96     25000



In [125]:
y_pred = clf2.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)

print("Test set accuracy score :", test_acc )
print("confusion matrix : \n",confusion_matrix(y_test, y_pred))
print("classification report :\n",classification_report(y_test, y_pred))

Test set accuracy score : 0.9262
confusion matrix : 
 [[21729  1318]
 [  527  1426]]
classification report :
              precision    recall  f1-score   support

      False       0.98      0.94      0.96     23047
       True       0.52      0.73      0.61      1953

avg / total       0.94      0.93      0.93     25000



# 7. Effect  of  regularization  constant C  on SVM  performance 
  C
∈ [0.1,10,1000,100000]

In [126]:
for c in [0.1, 10, 1000, 100000 ]:
    clf3 = svm.SVC(C=c)
    clf3.fit(X_train, y_train)
    
    y_pred = clf3.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)
    
    y_pred = clf3.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print("C : ", c,'\t', "train set accuracy : ", train_acc,"\ttest set accuracy : ", test_acc,"\n")
    

C :  0.1 	 train set accuracy :  0.95192 	test set accuracy :  0.9448 

C :  10 	 train set accuracy :  0.96176 	test set accuracy :  0.93616 

C :  1000 	 train set accuracy :  0.96248 	test set accuracy :  0.9262 

C :  100000 	 train set accuracy :  0.9644 	test set accuracy :  0.92496 



# 8.Logistic Regression

In [284]:
X = [feature_ABV_taste(d) for d in data]
y = [d['beer/style'] == 'American IPA' for d in data]

In [285]:
X = np.array(X)
y = np.array(y)

In [286]:
X_train, X_test = X[:int(len(X)/2)], X[int(len(X)/2):]

In [287]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]

In [295]:
def initializeTheta(X):
    n_coeff = X.shape[1]
    return np.zeros(n_coeff)

def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

def costFun(X, y, theta):
    m = X.shape[0]
    z = np.dot(X, theta)
    h = sigmoid(z)
    
    return (-1/m)*( np.sum( y * np.log(h) + (1 - y) * np.log (1 - h) ) )

def cal_gradient(X, y, theta):
    z = np.dot(X, theta)
    h = sigmoid(z)
    return np.dot( X.T, (h - y) ) / len(y)

def perform_logisticReg(X, y, lr=0.01, num_itr=100001):
    theta = initializeTheta(X)
    for i in range(num_itr):
        gradient = cal_gradient(X, y, theta)
        theta -= lr * gradient
        if( i % 10000 == 0):
            print("Loss  :{}\n".format(costFun(X, y, theta)))
    return theta

def predict_logisticReg(theta, X):
    z = np.dot(X, theta) 
    h = sigmoid(z)
    y_pred = np.round(h + 0.1)
    return y_pred

def accuracy(y_test, y_pred):
    correct_pred =[ act == pred for act, pred in zip(y_test, y_pred)] 
    accuracy =  np.sum(correct_pred) * 1.0 / len(correct_pred)
    return accuracy


In [296]:
theta = perform_logisticReg(X_train, y_train, lr=0.01, num_itr=100001)

Loss  :0.5671449053246294

Loss  :0.27006366173259144

Loss  :0.26909583942386867

Loss  :0.2685040375639266

Loss  :0.2681354550899628

Loss  :0.26790255770082716

Loss  :0.26775371057680347

Loss  :0.2676577193019152

Loss  :0.26759536926394356

Loss  :0.26755463794585554

Loss  :0.26752790701431667



In [297]:
print("theta : {}".format(theta) )

theta : [-1.62445469 -0.45498184  0.63841396]


In [298]:
y_pred = predict_logisticReg(theta, X_train)
print("train set accuracy : ", accuracy(y_train, y_pred) )

train set accuracy :  0.91272


In [299]:
y_pred = predict_logisticReg(theta, X_test)

print("test set accuracy : ", accuracy(y_test, y_pred) )

test set accuracy :  0.9208
